# Combine PM-2.5 and Satellite Data

In [ ]:
# To access Earth Engine API
import ee
ee.Authenticate()
ee.Initialize(project='ee-aspenjkmorgan')

### Import
* Montana border
* MODIS AOD
* NOAA RTMA variables
* NASA PBLH

In [ ]:
# Use Census data to define MT boundaries
MT = ee.FeatureCollection('TIGER/2018/States').filter(
  ee.Filter.eq('NAME', 'Montana')).geometry()

In [ ]:
# Import AOD data from MODIS MAIAC
modis_ic = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES')

# Import hourly weather data from noaa (2.5km)
# Cast to floats since there are some issues with the data
band_to_type = {
    'HGT': 'float', # Model terrain elevation
    'PRES': 'float', # Pressure
    'TMP': 'float', # Temperature
    'DPT': 'float', # Dew point temperature
    'UGRD': 'float', # u component of wind
    'VGRD': 'float', # v componenet of wind
    'SPFH': 'float', # specific humidity
    'WDIR': 'float', # Wind direction (from which its blowing)
    'WIND': 'float', # Wind speed
    'GUST': 'float', # Wind gust speed
    'VIS': 'float', # Visibility
    'TCDC': 'float', # total cloud cover
    'ACPC01': 'float' # Total precipitation
}
order = ['HGT','PRES', 'TMP', 'DPT', 'UGRD', 'VGRD', 'SPFH', 'WDIR', 'WIND', 'GUST',
        'VIS', 'TCDC', 'ACPC01']
noaa_col = ee.ImageCollection('NOAA/NWS/RTMA').cast(band_to_type, order)

# Select variables
noaa_col = noaa_col.select('PRES', 'TMP', 'DPT', 'WDIR', 'WIND')

# Import planetary boundary height
pblh_col = ee.ImageCollection('NASA/GSFC/MERRA/flx/2').select('PBLH')

In [ ]:
# This is PM-2.5 data from 2012-2022 using Terra and Aqua time windows
pm_one = ee.FeatureCollection('projects/ee-aspenjkmorgan/assets/PM_averaged/PM_ave_1')
pm_two = ee.FeatureCollection('projects/ee-aspenjkmorgan/assets/PM_averaged/PM_ave_2')
pm_data = ee.FeatureCollection([pm_one, pm_two]).flatten()

# # Get a unique list of all the datetimes in the PM2.5 collection
all_datetimes = pm_data.distinct('system:time_start')

### Reproject and put all data into table together
* Cloud mask data using bitwiseExtract helper function
* Calculate relative humidity

In [ ]:
# AOD cloud-mask helper method
def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)

# Helper method for masking, reprojecting, and overlapping data from each datetime
def forEachDatetime(dt_feature):
  # create average window for corresponding AOD data
  dt = ee.Date(dt_feature.get('system:time_start'))
  start = ee.Date(dt).advance(-1, 'hour')
  end = ee.Date(dt).advance(1, 'hour')

  # mask composite AOD image using AOD_QA band, reproject, select AOD, scale it
  modis_image = modis_ic.filterDate(start, end).mean()
  qa = modis_image.select('AOD_QA').toUint16()
  qa_cloud = bitwiseExtract(qa, 0, 2).eq(1)
  aod_qa = bitwiseExtract(qa, 5, 7).eq(0)
  mask = qa_cloud.And(aod_qa)
  aod_buf1 = modis_image.updateMask(mask)
  aod_buf2 = aod_buf1.reproject(**{
      'crs': ee.Projection('EPSG:4326'),
      'scale': 1000})
  aod_image = aod_buf2.select('Optical_Depth_047').multiply(0.001).rename('aod')

  # Filter NOAA RTMA weather collection and reproject
  noaa_image = noaa_col.filterDate(start, end).mean().reproject(**{
      'crs': ee.Projection('EPSG:4326'),
      'scale': 1000})

  # Filter NASA planetary boundary height data and reproject
  pblh_image = pblh_col.filterDate(start, end).mean().reproject(**{
      'crs': ee.Projection('EPSG:4326'),
      'scale': 1000})

  # Combine all raster data
  raster = ee.Image.cat(aod_image, noaa_image, pblh_image)

  # Select PM data for day
  pm_pts = pm_data.filterDate(start, end)

  # Create table that overlaps data
  inputs = raster.bandNames()
  dt_col = ee.FeatureCollection(raster.reduceRegions(**{
      'collection': pm_pts,
      'reducer': ee.Reducer.mean().forEachBand(raster),
      'crs': ee.Projection('EPSG:4326'),
      'scale': 1000}))
  return dt_col

# helper method for calculating relative humidity
# rel_hum = 100×(e^((17.625xdp)/(243.04+dp))/(e^((17.625xtemp)/(243.04+temp)))
def getRH(feature):
  dp = ee.Number(feature.get('DPT'))
  temp = ee.Number(feature.get('TMP'))
  c1 = ee.Number(17.625)
  c2 = ee.Number(243.04)
  rel_hum = ee.Number(100).multiply(
      ((dp.multiply(c1)).divide(dp.add(c2)).exp())
      .divide(((temp.multiply(c1)).divide(temp.add(c2))).exp()))
  return feature.set('RH', rel_hum)


In [ ]:
# define collection of feature collections then flatten
full_collection = ee.FeatureCollection(all_datetimes.map(forEachDatetime))
flat_collection = full_collection.flatten()

# remove null values
fil_collection = flat_collection.filter(ee.Filter.notNull(['AOD', 'DPT', 'PRES', 'TMP', 'WDIR', 'WIND', 'PBLH']))

# add relative humidity column
final_collection = fil_collection.map(getRH)

### Export in parts
* There is a limit to the number of features (5000) an ee.FeatureCollection can contain.

In [ ]:
# Break data up to export
cutoffs = {
    '2012': '2015',
    '2015': '2018',
    '2018': '2020',
    '2020': '2023'
    }

for start_year in cutoffs:
  part_col = final_collection.filterDate(ee.Date(start_year + '-1-1'),
                                       ee.Date(cutoffs[start_year] + '-1-1'))

  # export table to assests
  ee.batch.Export.table.toAsset(**{
    'collection': part_col,
    'description': 'Multivariable_Table_' + start_year + '-' + cutoffs[start_year],
    'assetId': 'projects/ee-aspenjkmorgan/assets/Data/Mul_Var_' + start_year + '-' + cutoffs[start_year]}).start()